In [20]:
# 패키지 임포트
import numpy as np
import pandas as pd
import scipy.stats as ss
import os 
import matplotlib.pyplot as plt
import seaborn as sns
# !pip install plotly
# import plotly.offline as off
# import plotly.graph_objs as go
pd.set_option('display.max_columns', 500)

plt.rc("font", family="Malgun Gothic") # matplotlib 한글깨짐 방지

In [29]:
train_data = pd.read_csv('train_data.csv', encoding='utf-8')
# 거래금액, 전용면적 데이터 컬럼명 변경
train_data.rename(columns = {'거래금액(만원)':'거래금액', '전용면적(㎡)':'전용면적'}, inplace=True)
# 거래금액 숫자형으로 변경
train_data['거래금액'] = train_data['거래금액'].apply(lambda x: x.replace(',', '')).astype('int64')

In [30]:
train_data

,시군구,단지명,전용면적,계약년월,거래금액,층,건축년도,도로명
0,서울특별시 강남구 개포동,개포6차우성아파트1동~8동,79.97,201801,130000,4,1987,언주로 3
1,서울특별시 강남구 개포동,개포6차우성아파트1동~8동,79.97,201801,117000,2,1987,언주로 3
2,서울특별시 강남구 개포동,개포6차우성아파트1동~8동,79.97,201801,130000,1,1987,언주로 3
3,서울특별시 강남구 개포동,개포6차우성아파트1동~8동,79.97,201803,139500,2,1987,언주로 3
4,서울특별시 강남구 개포동,개포6차우성아파트1동~8동,54.98,201804,107500,5,1987,언주로 3
...,...,...,...,...,...,...,...,...
240916,서울특별시 중랑구 중화동,한영(104),67.57,202008,26000,2,2003,동일로144길 74
240917,서울특별시 중랑구 중화동,현대휴앤미,95.94,202007,44000,6,2007,봉화산로27가길 23
240918,서울특별시 중랑구 중화동,현대휴앤미,100.17,202012,54800,5,2007,봉화산로27가길 23
240919,서울특별시 중랑구 중화동,현대휴앤미(102동),77.71,202009,40000,5,2007,봉화산로27가길 19


In [31]:
region_split = train_data['시군구'].str.split(' ', expand=True)
region_split.columns = ['si', 'gu', 'dong']

# train_data.drop('시군구', axis=1, inplace=True)

train_data = pd.concat([train_data, region_split], axis=1)
# train_data = train_data[['si', 'gu', 'dong', 'price']]
# train_data.drop('si', axis=1, inplace=True)
train_data

,시군구,단지명,전용면적,계약년월,거래금액,층,건축년도,도로명,si,gu,dong
0,서울특별시 강남구 개포동,개포6차우성아파트1동~8동,79.97,201801,130000,4,1987,언주로 3,서울특별시,강남구,개포동
1,서울특별시 강남구 개포동,개포6차우성아파트1동~8동,79.97,201801,117000,2,1987,언주로 3,서울특별시,강남구,개포동
2,서울특별시 강남구 개포동,개포6차우성아파트1동~8동,79.97,201801,130000,1,1987,언주로 3,서울특별시,강남구,개포동
3,서울특별시 강남구 개포동,개포6차우성아파트1동~8동,79.97,201803,139500,2,1987,언주로 3,서울특별시,강남구,개포동
4,서울특별시 강남구 개포동,개포6차우성아파트1동~8동,54.98,201804,107500,5,1987,언주로 3,서울특별시,강남구,개포동
...,...,...,...,...,...,...,...,...,...,...,...
240916,서울특별시 중랑구 중화동,한영(104),67.57,202008,26000,2,2003,동일로144길 74,서울특별시,중랑구,중화동
240917,서울특별시 중랑구 중화동,현대휴앤미,95.94,202007,44000,6,2007,봉화산로27가길 23,서울특별시,중랑구,중화동
240918,서울특별시 중랑구 중화동,현대휴앤미,100.17,202012,54800,5,2007,봉화산로27가길 23,서울특별시,중랑구,중화동
240919,서울특별시 중랑구 중화동,현대휴앤미(102동),77.71,202009,40000,5,2007,봉화산로27가길 19,서울특별시,중랑구,중화동


In [32]:
# 타겟설정
target = '거래금액' 

In [33]:
# 구 별 가격 평균 인코딩
gu_mean = train_data.groupby('gu')[target].mean()
# gu_mean
train_data['gu_mean'] = train_data['gu'].map(gu_mean)
train_data[['gu', 'gu_mean']]

,gu,gu_mean
0,강남구,172391.946333
1,강남구,172391.946333
2,강남구,172391.946333
3,강남구,172391.946333
4,강남구,172391.946333
...,...,...
240916,중랑구,44998.326330
240917,중랑구,44998.326330
240918,중랑구,44998.326330
240919,중랑구,44998.326330


In [34]:
# 동별 가격 평균 top30 리스트 추출
top30_dong = train_data.groupby('dong').mean().sort_values('거래금액', ascending=False)[:30].index

In [35]:
top30_dong

Index(['압구정동', '신문로2가', '반포동', '한남동', '대치동', '용산동5가', '서빙고동', '청담동', '도곡동',
       '잠원동', '주성동', '잠실동', '삼성동', '개포동', '청암동', '한강로3가', '홍파동', '하중동', '신천동',
       '평동', '회현동2가', '이촌동', '일원동', '여의도동', '신계동', '남대문로5가', '방배동', '보광동',
       '동빙고동', '한강로2가'],
      dtype='object', name='dong')

In [36]:
# top30에 들어가면 동 이름 그대로 아니면 기타로 매핑
def top30(x):
    if x in top30_dong:
        return x
    else:
        return '기타'

In [37]:
# 매핑한 결과로 새로운 컬럼 생성
train_data['top30_dong'] = train_data['dong'].apply(top30)

In [38]:
train_data

,시군구,단지명,전용면적,계약년월,거래금액,층,건축년도,도로명,si,gu,dong,gu_mean,top30_dong
0,서울특별시 강남구 개포동,개포6차우성아파트1동~8동,79.97,201801,130000,4,1987,언주로 3,서울특별시,강남구,개포동,172391.946333,개포동
1,서울특별시 강남구 개포동,개포6차우성아파트1동~8동,79.97,201801,117000,2,1987,언주로 3,서울특별시,강남구,개포동,172391.946333,개포동
2,서울특별시 강남구 개포동,개포6차우성아파트1동~8동,79.97,201801,130000,1,1987,언주로 3,서울특별시,강남구,개포동,172391.946333,개포동
3,서울특별시 강남구 개포동,개포6차우성아파트1동~8동,79.97,201803,139500,2,1987,언주로 3,서울특별시,강남구,개포동,172391.946333,개포동
4,서울특별시 강남구 개포동,개포6차우성아파트1동~8동,54.98,201804,107500,5,1987,언주로 3,서울특별시,강남구,개포동,172391.946333,개포동
...,...,...,...,...,...,...,...,...,...,...,...,...,...
240916,서울특별시 중랑구 중화동,한영(104),67.57,202008,26000,2,2003,동일로144길 74,서울특별시,중랑구,중화동,44998.326330,기타
240917,서울특별시 중랑구 중화동,현대휴앤미,95.94,202007,44000,6,2007,봉화산로27가길 23,서울특별시,중랑구,중화동,44998.326330,기타
240918,서울특별시 중랑구 중화동,현대휴앤미,100.17,202012,54800,5,2007,봉화산로27가길 23,서울특별시,중랑구,중화동,44998.326330,기타
240919,서울특별시 중랑구 중화동,현대휴앤미(102동),77.71,202009,40000,5,2007,봉화산로27가길 19,서울특별시,중랑구,중화동,44998.326330,기타


In [39]:
# top30 컬럼의 동 별 가격 평균 인코딩
dong_mean = train_data.groupby('top30_dong')[target].mean()
# dong_mean
train_data['dong_mean'] = train_data['top30_dong'].map(dong_mean)
train_data[['dong', 'dong_mean']]

,dong,dong_mean
0,개포동,166112.063772
1,개포동,166112.063772
2,개포동,166112.063772
3,개포동,166112.063772
4,개포동,166112.063772
...,...,...
240916,중화동,66458.989444
240917,중화동,66458.989444
240918,중화동,66458.989444
240919,중화동,66458.989444


In [40]:
train_data

,시군구,단지명,전용면적,계약년월,거래금액,층,건축년도,도로명,si,gu,dong,gu_mean,top30_dong,dong_mean
0,서울특별시 강남구 개포동,개포6차우성아파트1동~8동,79.97,201801,130000,4,1987,언주로 3,서울특별시,강남구,개포동,172391.946333,개포동,166112.063772
1,서울특별시 강남구 개포동,개포6차우성아파트1동~8동,79.97,201801,117000,2,1987,언주로 3,서울특별시,강남구,개포동,172391.946333,개포동,166112.063772
2,서울특별시 강남구 개포동,개포6차우성아파트1동~8동,79.97,201801,130000,1,1987,언주로 3,서울특별시,강남구,개포동,172391.946333,개포동,166112.063772
3,서울특별시 강남구 개포동,개포6차우성아파트1동~8동,79.97,201803,139500,2,1987,언주로 3,서울특별시,강남구,개포동,172391.946333,개포동,166112.063772
4,서울특별시 강남구 개포동,개포6차우성아파트1동~8동,54.98,201804,107500,5,1987,언주로 3,서울특별시,강남구,개포동,172391.946333,개포동,166112.063772
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
240916,서울특별시 중랑구 중화동,한영(104),67.57,202008,26000,2,2003,동일로144길 74,서울특별시,중랑구,중화동,44998.326330,기타,66458.989444
240917,서울특별시 중랑구 중화동,현대휴앤미,95.94,202007,44000,6,2007,봉화산로27가길 23,서울특별시,중랑구,중화동,44998.326330,기타,66458.989444
240918,서울특별시 중랑구 중화동,현대휴앤미,100.17,202012,54800,5,2007,봉화산로27가길 23,서울특별시,중랑구,중화동,44998.326330,기타,66458.989444
240919,서울특별시 중랑구 중화동,현대휴앤미(102동),77.71,202009,40000,5,2007,봉화산로27가길 19,서울특별시,중랑구,중화동,44998.326330,기타,66458.989444


In [41]:
# minmax 정규화
from sklearn.preprocessing import MinMaxScaler

In [42]:
# gu_mean의 minmax 정규화
minmax_scaler = MinMaxScaler()
gu_mean_df = train_data[['gu_mean']]
minmax_scaled_gu_mean = pd.DataFrame(minmax_scaler.fit_transform(gu_mean_df), columns=['gu_mean'])
train_data['gu_mean'] = minmax_scaled_gu_mean

In [43]:
# dong_mean의 minmax 정규화
# minmax_scaler = MinMaxScaler()
dong_mean_df = train_data[['dong_mean']]
minmax_scaled_dong_mean = pd.DataFrame(minmax_scaler.fit_transform(dong_mean_df), columns=['dong_mean'])
train_data['dong_mean'] = minmax_scaled_dong_mean

In [46]:
'''
si, gu, dong, gu_mean, top_30_dong, dong_mean 컬럼 생성
'''

train_data

,시군구,단지명,전용면적,계약년월,거래금액,층,건축년도,도로명,si,gu,dong,gu_mean,top30_dong,dong_mean
0,서울특별시 강남구 개포동,개포6차우성아파트1동~8동,79.97,201801,130000,4,1987,언주로 3,서울특별시,강남구,개포동,1.000000,개포동,0.458209
1,서울특별시 강남구 개포동,개포6차우성아파트1동~8동,79.97,201801,117000,2,1987,언주로 3,서울특별시,강남구,개포동,1.000000,개포동,0.458209
2,서울특별시 강남구 개포동,개포6차우성아파트1동~8동,79.97,201801,130000,1,1987,언주로 3,서울특별시,강남구,개포동,1.000000,개포동,0.458209
3,서울특별시 강남구 개포동,개포6차우성아파트1동~8동,79.97,201803,139500,2,1987,언주로 3,서울특별시,강남구,개포동,1.000000,개포동,0.458209
4,서울특별시 강남구 개포동,개포6차우성아파트1동~8동,54.98,201804,107500,5,1987,언주로 3,서울특별시,강남구,개포동,1.000000,개포동,0.458209
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
240916,서울특별시 중랑구 중화동,한영(104),67.57,202008,26000,2,2003,동일로144길 74,서울특별시,중랑구,중화동,0.030539,기타,0.000000
240917,서울특별시 중랑구 중화동,현대휴앤미,95.94,202007,44000,6,2007,봉화산로27가길 23,서울특별시,중랑구,중화동,0.030539,기타,0.000000
240918,서울특별시 중랑구 중화동,현대휴앤미,100.17,202012,54800,5,2007,봉화산로27가길 23,서울특별시,중랑구,중화동,0.030539,기타,0.000000
240919,서울특별시 중랑구 중화동,현대휴앤미(102동),77.71,202009,40000,5,2007,봉화산로27가길 19,서울특별시,중랑구,중화동,0.030539,기타,0.000000


In [76]:
# train_data.to_csv('temp.csv')